In [ ]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import random
from numpy.linalg import inv

In [ ]:
connection = psycopg2.connect(dbname='balance909', user='balance909', password='Sa_909_wuf;', host='89.108.115.118', port='5432')
cursor = connection.cursor()

In [ ]:
# Ищем Y_i
cursor.execute("""
SELECT 
  company_id, 
  sum(count_product * price)
FROM consumer_requirement_base cr
JOIN product p ON p.id = cr.product_id
GROUP BY company_id
ORDER BY company_id;
""")

Y_i = pd.DataFrame(data=cursor.fetchall(), dtype=np.int64, columns=['i', 'Y_i'])
Y_i

,i,Y_i
0,1,108350
1,2,1000000
2,3,25000000
3,4,999990
4,5,9999000
5,6,50
6,7,30000000
7,8,166315
8,9,34327300
9,10,774150


In [ ]:
# Определим количество представленных компаний
cursor.execute("""
SELECT count(*)
FROM company;
""")
company_count = cursor.fetchall()[0][0]
company_count

14

In [ ]:
# Ищем значения X_ij
X_ij = np.zeros((company_count, company_count), dtype=np.int64)

for i in range(company_count):
  cursor.execute("""
  SELECT 
    cr.company_id, 
    sum(count_product * price) 
  FROM company_requirement cr 
  JOIN product p on p.id = cr.product_id 
  WHERE p.company_id = %d
  GROUP by cr.company_id
  ORDER by cr.company_id;
  """%(i+1))

  x_ij = pd.DataFrame(data=cursor.fetchall(), dtype=np.int64, columns=['j', 'X_ij'])
  for index, row in x_ij.iterrows():
    X_ij[i, row.j-1] = row.X_ij

X_ij = pd.DataFrame(data=X_ij, dtype=np.int64)
X_ij

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,4000,0,0,0,0,0,0,1036000,25000,0,0,0,0
1,100000,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1000000,0,0,0,0,0,0,0,0,0,0,0
3,199998,0,999990,4999950,0,0,0,0,0,0,0,0,99999,0
4,150000,0,0,0,1500000,0,0,0,0,0,0,0,0,0
5,100,0,0,0,0,8050,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,30000000,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,269700,0,0,0,0,0,0
8,0,350000,35000,0,0,0,0,0,35000000,7000,0,0,0,0
9,0,195000,0,0,0,0,0,0,0,780000,0,0,0,0


In [ ]:
# Вычисляем X_i  = Σ_j X_ij + Y_i
X_i = np.zeros((company_count))
for i in range(company_count):
  X_i[i] = Y_i.Y_i[i]
  for j in range(company_count):
    X_i[i] += X_ij[i][j]

X_i = pd.DataFrame(data=X_i, dtype=np.int64, columns=['X_i'])
X_i

,X_i
0,560948
1,1556500
2,27034990
3,5999940
4,11499000
5,8100
6,60000000
7,436015
8,70373300
9,1596150


In [ ]:
# Коеффициенты матрицы A: A_ij = X_ij / X_j
A = np.zeros((company_count, company_count))

for i in range(company_count):
  for j in range(company_count):
    A[i, j] = X_ij[i][j] / X_i.X_i[j]

A = pd.DataFrame(data=A, dtype=np.float64)
A

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000000,0.064247,0.000000,0.033333,0.013045,0.012346,0.0,0.000000,0.000000,0.000000,0.000167,0.0,0.0,0.000000
1,0.007131,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.004973,0.122169,0.000500,0.0,0.0,0.000000
2,0.000000,0.000000,0.036989,0.166667,0.000000,0.000000,0.0,0.000000,0.000497,0.000000,0.000000,0.0,0.0,0.000000
3,0.000000,0.000000,0.000000,0.833333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.130446,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.993827,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.618557,0.000000,0.000000,0.000000,0.0,0.0,0.000000
8,1.846874,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.497348,0.000000,0.000667,0.0,0.0,0.000000
9,0.044567,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000099,0.488676,0.000667,0.0,0.0,0.000000


In [ ]:
# (E-A)^-1
E = np.eye(company_count)
M = inv(E - A)
M = pd.DataFrame(data=M, dtype=np.float64)
M

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.002327,0.064396,-0.00000,0.200465,1.503641e-02,2.004655,-0.0,-0.000000,0.000640,1.538598e-02,4.199016e-04,-0.0,-0.0,-0.0
1,0.036224,1.002327,0.00000,0.007245,5.434202e-04,0.072449,0.0,0.000000,0.009965,2.394827e-01,1.347089e-03,0.0,0.0,0.0
2,0.001902,0.000122,1.03841,1.038790,2.853269e-05,0.003804,0.0,0.000000,0.001029,2.919601e-05,2.166820e-06,0.0,0.0,0.0
3,0.000000,0.000000,0.00000,6.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0
4,0.000000,0.000000,0.00000,0.000000,1.150015e+00,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0
5,-0.000000,-0.000000,-0.00000,-0.000000,-1.405126e-16,162.000000,-0.0,-0.000000,-0.000000,-1.229390e-16,4.083984e-18,-0.0,-0.0,-0.0
6,0.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000,2.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0
7,0.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.0,2.621622,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0
8,3.682808,0.236608,0.00000,0.736562,5.524764e-02,7.365616,0.0,0.000000,1.991799,5.653203e-02,4.195599e-03,0.0,0.0,0.0
9,0.088080,0.005659,0.00000,0.017616,1.321333e-03,0.176160,0.0,0.000000,0.000443,1.957059e+00,2.645198e-03,0.0,0.0,0.0


In [ ]:
Y_ = np.array(random.sample(range(0, 1000000), company_count))
X_ = np.matmul(M, Y_)
X_ = pd.DataFrame(data=X_, dtype=np.float64)
X_.columns = [''] * len(X_.columns)
X_

,
0,1.893253e+06
1,1.111463e+06
2,9.819760e+05
3,2.993076e+06
4,1.111350e+06
5,9.876703e+07
6,1.175360e+06
7,1.434551e+05
8,8.212676e+06
9,1.098350e+06
